In [1]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 4.4 MB/s 
     |████████████████████████████████| 662 kB 11.6 MB/s 
     |████████████████████████████████| 88 kB 4.0 MB/s 
     |████████████████████████████████| 85 kB 2.5 MB/s 
     |████████████████████████████████| 5.2 MB 14.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 5.8 MB 67.1 MB/s 
     |████████████████████████████████| 1.3 MB 72.8 MB/s 
     |████████████████████████████████| 182 kB 84.4 MB/s 
     |████████████████████████████████| 7.6 MB 39.1 MB/s 
     |████████████████████████████████| 1.1 MB 46.9 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700861 sha256=feb755c17979ae011bc0f9ebeeae0e3978883698db5299e0250e43a86bed3c10
  Stored in directory: /root/.cache/

In [2]:
!pip install konlpy pecab gensim
!curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 26.4 MB 1.3 MB/s 
     |████████████████████████████████| 465 kB 9.7 MB/s 
     |████████████████████████████████| 131 kB 47.8 MB/s 
  Created wheel for pecab: filename=pecab-1.0.2-py3-none-any.whl size=26646461 sha256=7253e5bb5a73790738872a3cf0b83ece8f08892a533c96405e603270b2ec5895
  Stored in directory: /root/.cache/pip/wheels/57/e2/87/fe894d7193d71e94506f6bb86affe21bdfdebe8f6f2b665a1c
Successfully built pecab
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_aut

In [3]:
import pandas as pd
from tqdm.notebook import tqdm
# 형태소 분석기 불러오기
from konlpy.tag import Mecab; mecab = Mecab()
# CountVectorizer 불러오기 -> document-term matrix를 만들어주는 함수.
from sklearn.feature_extraction.text import CountVectorizer
# bertopic 불러오기
from bertopic import BERTopic

In [5]:
file_name = input("텍스트 파일 이름을 입력하세요 : ")
start_date = input("시작 날짜를 입력하세요 : ")
end_date = input("마지막 날짜를 입력하세요 : ")

# Colab Notebooks
base_path = "/content/drive/MyDrive/Colab Notebooks/data/topic/"

# 아까 크롤링한 파일
## 크롤링한 파일이 바뀌면, 경로도 수정되어야 합니다.
data_path = f"{file_name}_({start_date}~{end_date}).xlsx"

텍스트 파일 이름을 입력하세요 : 갤럭시
시작 날짜를 입력하세요 : 2022.01.01
마지막 날짜를 입력하세요 : 2022.12.06


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/topic/갤럭시_(2022.01.01~2022.12.06).xlsx')

contents = data['contents'].dropna()
contents

0       언론인 출신 백수현, 박승희 부사장도 나란히 승진    ▲ 삼성전자 이영희 사장\t...
1       이영희 DX부문 글로벌마케팅실장 사장  삼성전자가 5일 네트워크와 반도체 사업의 성...
2       삼성 새 역사 쓴 이영희 사장마케팅 전문가로 2007년 영입유니레버·SC존슨·로레알...
3       이영희 삼성전자 DX부문 글로벌마케팅센터장. 연합뉴스삼성에서 오너가 출신이 아닌 첫...
4       비오너가 출신 이영희 삼성전자 사장…2012년 부사장 승진 이후 10년 만2007년...
                              ...                        
2717    이통3사 '갤S22' 공시지원금 5.3만~24만원…전작 대비 절반 수준실구매자는 2...
2718    갤럭시 S20·노트10·노트9 15만 원 추가 보상    전국 대형마트 및 통신사 ...
2719    [이데일리 전재욱 기자] 온라인 장보기 업체 마켓컬리는 삼성전자 신작 스마트폰 ‘갤...
2720    이통3사 '갤S22' 공시지원금 5.3만~24만원…전작 대비 절반 수준실구매자는 2...
2721    갤럭시 S20·노트10·노트9 15만 원 추가 보상    전국 대형마트 및 통신사 ...
Name: contents, Length: 2722, dtype: object

In [8]:
p_contents = []
for doc in tqdm(contents):
    tokens = [token for token in mecab.morphs(doc) if len(token) > 1]
    p_contents.append(" ".join(tokens))

print(p_contents[0])

  0%|          | 0/2722 [00:00<?, ?it/s]

언론인 출신 백수현 박승희 부사장 나란히 승진 삼성전자 이영희 사장 삼성전자 이영희 부사장 사장 으로 승진 시키 사장 승진 인사 단행 삼성 에서 오너 일가 아닌 여성 사장 나온 이번 처음 이영희 사장 그동안 삼성전자 에서 가장 유력 최초 여성 사장 후보 거론 돼왔 유니레버 로레알 출신 마케팅 전문가 2007 삼성전자 마케팅 담당 임원 으로 발탁 2012 부사장 으로 승진 10 부사 장직 유지 신임 사장 갤럭시 시리즈 마케팅 성공 으로 이끌 삼성전자 글로벌 브랜드 인지도 끌어올리 는데 기여 다는 평가 는다 삼성 역량 성과 여성 부사장 사장 으로 승진 시켜 여성 인재 에게 성장 비전 제시 과감히 도전 계기 마련 밝혔 이재용 삼성전자 회장 평소 성별 국적 불문 세상 바꿀 인재 모셔오 양성 해야 한다 강조 회장 취임 으로 우리 놓인 현실 엄중 시장 냉혹 지만 우리 어떻게 대응 느냐 따라 기회 어렵 힘들 일수록 앞서 준비 실력 키워 나가 한다 창업 이래 가장 중시 가치 인재 기술 성별 국적 불문 세상 바꿀 인재 모셔오 양성 해야 한다 세상 기술 투자 해야 한다 다음 사장 승진 명단 삼성전자 DX 부문 네트워크 사업 전략 마케팅 팀장 김우준 부사장 삼성전자 DX 부문 네트워크 사업 부장 사장 삼성전자 DX 부문 글로벌 마케팅 센터 이영희 부사장 삼성전자 DX 부문 글로벌 마케팅 실장 사장 삼성전자 DS 부문 글로벌 제조 인프라 총괄 남석우 부사장 삼성전자 글로벌 제조 인프라 총괄 제조 담당 사장 삼성전자 DS 부문 반도체 연구 소장 송재혁 부사장 삼성전자 DS 부문 CTO 반도체 연구 소장 사장 삼성전자 DX 부문 커뮤니케이션 팀장 백수현 부사장 삼성전자 DX 부문 커뮤니케이션 팀장 사장 삼성물산 건설 부문 커뮤니케이션 팀장 박승희 부사장 삼성전자 CR Corporate Relations 담당 사장 삼성전자 중국 전략 협력 부실 양걸 부사장 삼성전자 중국 전략 협력 실장 사장 위촉 업무 변경 삼성전자 DX 부문 네트워크 사업 부장 전경훈 사장 삼성전자 DX 부문 CTO 

Source : https://www.sbert.net/docs/pretrained_models.html

In [9]:
vectorizer = CountVectorizer(max_features=3000) # 빈도가 높은 3000개만 사용.

# sentence-transformer에 있는 pretrained model 사용 가능.
model = BERTopic(embedding_model='paraphrase-multilingual-mpnet-base-v2', # pretrained-model
                 vectorizer_model=vectorizer, # vectorizer
                 nr_topics=5,  # num_topics
                 top_n_words=30, # top_n_words
                 calculate_probabilities=True)

In [10]:
topics, probs = model.fit_transform(p_contents) # dtm

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [12]:
# Intertopic Distance Map
model.visualize_topics()

In [13]:
# Topic Probability Distribution
model.visualize_distribution(probs[0], min_probability=0.0001)

In [14]:
# Show top words
model.visualize_barchart(top_n_topics=10, n_words=10)

In [15]:
# Print each topic info
for i in range(10):
    print(f'{i}번째 토픽 : {model.get_topic(i)}')

0번째 토픽 : [('보상', 0.11539569384184713), ('고객', 0.11362433192535212), ('노트', 0.08653102309331397), ('통신사', 0.07430438855085637), ('구매', 0.06665585262217515), ('10', 0.06312781765677401), ('팃은', 0.06216286483485335), ('atm', 0.061871901581150794), ('프라자', 0.061188090445076115), ('설치', 0.06079592962925307), ('프로그램', 0.060317672457473694), ('추가', 0.06017677576538595), ('갤럭시', 0.06005063380217049), ('디지털', 0.059565815637650864), ('자급', 0.05884511858649947), ('20', 0.058731089459171604), ('이벤트', 0.05767607130978789), ('제공', 0.05055018078024274), ('한다', 0.04803314889367283), ('삼성', 0.04796003874177452), ('까지', 0.04721288288129553), ('이마트', 0.045065536026866186), ('찾아가', 0.04501453266749015), ('홍대', 0.04490741599632339), ('스페셜', 0.044793343548532004), ('본점', 0.04470463019494544), ('신규', 0.044595423078303206), ('매장', 0.04454183380596342), ('운영', 0.04383623655372424), ('확인', 0.04361785543856976)]
1번째 토픽 : [('컬리', 0.12534725168999777), ('마켓', 0.12504170530345213), ('갤럭시', 0.10000727985284605), ('2

#### References

1. Bertopic documentation : https://maartengr.github.io/BERTopic/index.html

2. Bertopic github : https://github.com/MaartenGr/BERTopic

3. 한국어 버토픽(Korean BERTopic) : https://wikidocs.net/162079